(tmux_pair-programming)=
# Парное программирование с tmux

До сих пор вы вносили изменения в конфигурацию и учились работать с tmux на своем собственном компьютере.
Но одно из самых популярных применений tmux разработчиками — парное программирование.
Это было на самом деле мое первое знакомство с tmux, и я сразу увидел потенциал, когда мой друг показал мне различные его функции.

Парное программирование обладает рядом преимуществ.
Работа с другим разработчиком может помочь вам увидеть то, что вы могли бы и не увидеть сами, но если вы физически не находитесь в одном и том же месте, парное программирование может быть несколько трудным.
Совместное использование экрана через iChat, Skype или даже GoToMeeting широкой пропускной способности сети и может быть затруднено, если вы используете не самое лучшее сетевое подключение.
В этой главе мы рассмотрим использование tmux для парного программирования, чтобы вы могли удаленно работать с другим разработчиком даже при самом медленном подключении Wi-Fi в отеле.

Существует два способа работы с удаленными пользователями. Первый способ предполагает создание новой учетной записи пользователя, общей для всех.
Вы настраиваете tmux и свою среду разработки под этой учетной записью и используете ее как общее рабочее пространство.
Второй подход использует сокеты tmux, поэтому у вас может быть второй пользователь подключающийся к сеансу tmux без необходимости иметь одну общуюю учетную запись.

Оба этих метода имеют врожденный недостаток безопасности: они позволяют кому-то другому видеть что-то на вашем экране и в вашем аккаунте.
Вы приглашаете кого-то потенциально просмотреть ваши файлы. Чтобы обойти это, разумно использовать промежуточный сервер для соединения.
Используя недорогой VPS или виртуальную машину с VirtualBox[12] и Vagrant[13], вы можете быстро создать среду разработки для сопряжения.
В этой главе мы будем работать с удаленным сервером, поскольку мы рассмотрим оба этих подхода.

## Сопряжение с общей учетной записью
Использование общей учетной записи — самый простой способ работать с другим пользователем.
Проще говоря, вы включаете SSH чтобы получить доступ к машине, которая будет выступать в качестве хоста, устанавливаете и настраиваете tmux на этой машине, а затем создаете там сеанс tmux.
Второй пользователь входит в эту машину с той же учетной записью и подключается к сеансу.
Используя открытые ключи SSH, вы можете сделать процесс входа в систему еще проще.
Давай рассмотрим процесс настройки.
В этом примере мы будем использовать сервер с именем `puzzles` под управлением Ubuntu, на котором установлен SSH демон.

Сначала создайте пользователя «tmux» на сервере.
Это пользователь, которого все будут использовать для подключения к спаренной сессии.
На удаленном сервере выполните эту команду:

`tmux@puzzles$ adduse rtmux`

Мы хотим настроить учетную запись, чтобы мы могли получать ключи SSH от других разработчиков и использовать их для регистрации в этом аккаунте.
Для этого нужно создать файл `~/.ssh/authorized_keys` под учетной записью `tmux`.
Затем, используйте команду `su` для переключения на пользователя:

`tmux@puzzles$ su t mux`

Затем создайте папку `.ssh` и файл `.ssh/authorized_keys`, установив соответствующие разрешения.
Только `tmux` пользователю должно быть разрешено читать, писать или выполнять папку и файл.

```
tmux@puzzles$ mkdir ~/.ssh
tmux@puzzles$ touch ~/.ssh/authorized_keys
tmux@puzzles$ chmod 700 ~/.ssh
tmux@puzzles$ chmod 600 ~/.ssh/authorized_keys
```

Каждому пользователю, которого вы хотите подключить, нужен открытый ключ, который он сгенерирует на своем локальном компьютере.
Для генерации ключа, используйте команду

`$ ssh-keygen`

и следуйте подсказкам на экране.

Затем каждый пользователь передаст свой открытый ключ на сервер и добавит его в файл `authorized_keys`.
Есть несколько способов сделать это, но наиболее универсальным подходом было бы использование `cat` и `ssh` для передачи ключа и одновременной записи его в author_keys, например:


```
$ cat ~/.ssh/id_rsa.pub | ssh tmux@your_server 'cat >> .ssh/authorized_keys'
```

Вам будет предложено ввести пароль пользователя tmux, прежде чем вы сможете подключиться.
Команда `ssh-copy-id` немного упрощает этот процесс.
Если вы установите эту команду, используя свой пакетный менеджер на вашем клиенте, то вы можете передать ключ так:

```$ ssh-copy-id ​tmux@your_server```

Это автоматически скопирует файл `.id_rsa.pub`.

Данный процесс необходимо повторить  для всех пользователей, с которыми вы хотите поделиться этой учетной записью.

Далее на удаленном сервере  вам нужно настроть tmux, текстовые редакторы, компиляторы, языки программирования и систему управления версиями точно так же, как и в любой другой среде разработки.

После этого можно создаь новый tmux сеанс  на сервере

`tmux@puzzles$ tmux new-session -s Pairing`

Другой член вашей команды может войти на тот же компьютер и подключиться к сеансу с помощью команды:

`tmux@puzzles$ tmux attach -t Pairing`

Теперь вы можете работать над проектом совместно .
Более того, вы можете отсоединиться от сеанса и повторно подключиться к нему позже, а это означает, что вы можете оставить свою среду работающей на несколько дней или даже недель без перерыва.
У вас будет постоянная среда разработки, в которую вы можете войти из любого места, где есть терминал с поддержкой SSH.

## Использование общей учетной записи и групповых сеансов

Когда два человека подключены к одному и тому же сеансу tmux, они обычно оба видят одно и то же и взаимодействуют с одними и теми же окнами.
Но бывают случаи, когда полезно, чтобы один человек мог работать в другом окне, не беря полностью на себя управление.

Используя «групповые сеансы» (grouped sessions), вы можете сделать именно это.
Давайте продемонстрируем это, создав новый сеанс на нашем
удаленный сервер под названием `groupedsession`.

`tmux@puzzles$ tmux new-session -s groupedsession`

Затем, вместо присоединения к сеансу, другой пользователь может присоединиться к этому сеансу, *создав новый сеанс*, указав цель исходного сеанса, `groupedsession`, а затем указав свое собственное имя сеанса, например:
```
tmux@puzzles$ tmux new-session -t groupedsession -s mysession
```

Когда запустится второй сеанс, оба пользователя могут взаимодействовать с сеансом одновременно, как если бы второй пользователь присоединился к сеансу.
Однако пользователи могут создавать окна независимо друг от друга.
Итак, если наш новый пользователь создаст окно, вы оба увидите новое окно в строке состояния, но вы останетесь в окне, в котором сейчас работаете!
Это отлично подходит для случая «Эй, дай мне просто попробовать что-то», или когда один человек хочет использовать Emacs, а другой предпочитает Vim:

<!-- TODO
![](./img/-084-020.png)
-->

Второй пользователь может завершить свой сеанс с помощью команды `kill-session`, а оригинал все равно будет существовать.
Однако оба сеанса будут закрыты, если все окна будут закрыты, так что будьте осторожны!

Это кропотливая и сложная работа, если вы просто хотите, чтобы кто-то присоединился и помог вам с кодом.
Итак, давайте рассмотрим простую альтернативу, которая почти не требует времени для настройки.




## Быстрое сопряжение с tmate

tmate[14] — это форк tmux, предназначенное для безболезненного парного программирования.
Используя tmate, вы можете быстро пригласить другого разработчика к сотрудничеству.
Когда вы запускаете tmate, он генерирует адрес, который ваша пара может использовать для подключения.
Вам не нужно настраивать какие-либо ключи или использовать какие-либо промежуточные сервисы.
Вместо этого серверы tmate обрабатывают туннелирование соединения за вас.

Загвоздка в том, что вам нужно установить tmate и использовать его вместо tmux.
Но не волнуйтесь; он полностью поддерживает конфигурацию, которую вы уже создали.
Давайте посмотрим, как его установить.

В Ubuntu вы можете установить его, добавив PPA-репозиторий tmate в менеджер пакетов:

```
$ sudo apt-get install software-properties-common
$ sudo add-apt-repository ppa:tmate.io/archive
$ sudo apt-get update && sudo apt-get install tmate
```

На Mac вы можете установить его с помощью Homebrew:
` $ brew install tmate`

Как только tmate будет установлен, запустите его с помощью

`$ tmate`

и tmata запустится, отображая адрес подключения в нижней части окна, где будет находиться ваша строка состояния.

![](./img/-086-021.png)

Скопируйте этот адрес и отправьте своим напарникам, тогда они смогут сравз к вам присоединиться.
Если адрес исчезнет, прежде чем вы сможете его скопировать, или вы хотите увидеть его снова, выполните команду

`$ tmate show-messages`

для повторного просмотра адреса вместе с некоторыми другими интересными деталями, включая адрес только для чтения, который вы можете отправьте кому-нибудь, если вы просто хотите что-то продемонстрировать и не хотите, чтобы они имели возможность управления:

```
Sun Sep 25 17:46:13 2016 [tmate] Connecting to ssh.tmate.io...
Sun Sep 25 17:46:13 2016 [tmate] Note: clear your terminal before sharing readonly
access
Sun Sep 25 17:46:13 2016 [tmate] web session read only: https://...
Sun Sep 25 17:46:13 2016 [tmate] ssh session read only: ssh ...
Sun Sep 25 17:46:13 2016 [tmate] web session: https://...
Sun Sep 25 17:46:13 2016 [tmate] ssh session: ssh ...
```

tmate поддерживает те же команды, что и tmux, поэтому вы можете создавать именованные сеансы и даже скрипты для ваших конфигураций.
Вы даже можете использовать его с Tmuxinator, добавив следующее в свой Tmuxinator YAML-файл:

```
tmux_options: - S /tmp/your_project_name_tmate_socket
tmux_command: tmate
```

Поскольку tmate создает файл сокета со случайным именем, мы просто говорим ему не делать этого, передав ключ -S.
Затем мы говорим Tmuxinator, что он должен использовать tmate вместо tmux.

**Использование tmate на вашем собственном сервере**

Если вам неудобно переходить на http://ssh.tmate.io для подключения к другим сеансам, вы можете найти инструкции по настройке вашего собственного сервера на сайте tmate.[15]
Он предоставляет вам сервер, который вы компилируете и устанавливаете на свой собственный Linux сервер.
Затем вы запускаете сервер и настраиваете клиентские машины для использования этого сервера вместо службы по умолчанию.
Это может добавить больше безопасности, но вам нужно подумать о избыточности и доступности.
Например, адрес tmate.io распределен на несколько внутренних серверов, обеспечивающих высокую доступность.
Если вы хотите обеспечить непрерывность, вам нужно настроить свою среду подобным способом.

Использовать общие учетные записи или tmate легко, но не всегда желательно делиться учетными записями пользователей с членами команды или разрешать кому-либо удаленный доступ к вашей машине для разработки.
Посмотрим на альтернативный подход.

## Сопряжение с отдельными учетными записями и сокетами

Используя поддержку сокетов в tmux, вы можете создавать сеансы, к которым могут легко подключаться несколько пользователей.

Чтобы проверить это, создайте две новые учетные записи пользователей для сеанса: одну с именем «ted», а другую с именем
«barney».

`tmux@puzzles$ sudo adduser ted`

`tmux@puzzles$ sudo adduser barney `


Затем создайте группу «tmux» и папку `/var/tmux`, в которой будут храниться общие сеансы.

`tmux@puzzles$ sudo addgroup tmux`

`tmux@puzzles$ sudo mkdir /var/tmux`

Затем измените групповое владение папкой `/var/tmux`, чтобы группа tmux имела доступ:

`tmux@puzzles$ sudo chgrp tmux /var/tmux`

И измените права доступа к папке, чтобы новые файлы были доступны для всех участников группы  tmux.
группа:

`tmux@puzzles$ sudo chmod g+ws /var/tmux`

Наконец, добавьте `ted` и `barney` в группу tmux.

`tmux@puzzles$ sudo usermod -aG tmux ted`

`tmux@puzzles$ sudo usermod -aG tmux  barney`

Теперь давайте посмотрим, как эти пользователи могут работать вместе над проектом.

###  Создание и совместное использование сеансов

До сих пор вы использовали команду `new-session` для создания сеансов, но при этом используется расположение сокета по умолчанию, которое будет доступно не каждому пользователю. Вместо создания именованных сеансов мы создаем наши сеансы с помощью переключателя -S.

Войдите на свой сервер как `ted` и создайте новый сеанс tmux, используя файл сокета в папке `/var/tmux/`:

`ted@puzzles$ tmux -S /var/tmux/pairing`

В другом окне терминала войдите в систему как `barney`, а затем подключитесь к сеансу.
Но вместо указания сеанса с помощью ключа -t укажите расположение файла сокета, например:

`barney@puzzles$ tmux -S /var/tmux/pairinga ttach`

Теперь пользователь `barney` подключен к сеансу tmux и видит все, что видит пользователь `ted`.

Важно отметить, что при использовании этого подхода используется тот файл .tmux.conf, который запустил сеанс.
Наличие двух отдельных учетных записей не означает, что каждая учетная запись может использовать свои собственные файлы конфигурации в сеансе tmux, но это означает, что они могут настраивать свои учетные записи для других целей и каждый может инициировать свой собственный сеанс tmux по мере необходимости. Что еще более важно, это не позволяет `barney` попасть в домашний каталог `ted`.


## Что дальше?

Теперь, когда вы знаете, как использовать tmux, чтобы поделиться своим экраном с другими, вы можете использовать его для удаленного обучения, совместной работы над проектами с открытым исходным кодом или даже презентации.

Кроме того, вы можете использовать эту технику для запуска сеанса tmux на одном из ваших рабочих серверов, запускать инструментов мониторинга или консоли, а затем отсоединяться от них, оставляя эти инструменты работать в фоновом режиме.
Затем вы просто подключаетесь к своей машине, снова подключаетесь к сеансу, и все возвращается на прежнее место.
Я делаю что-то подобное со своей средой разработки.
Я установил tmux на VPS, что позволяет мне не использовать
ничего, кроме iPad, SSH-клиента и Bluetooth-клавиатуры для работы над кодом, когда меня нет дома.
Он прекрасно работает даже в сети 3G.

Парное программирование и удаленная работа — это всего лишь два примера того, как включение tmux в ваш рабочий процесс может сделать вас более продуктивным.
В следующей главе мы рассмотрим другие улучшения, которые мы можем сделать в нашей среде, когда изучаем продвинутые способы работы с окнами, панелями и нашей системой в целом.

## Для дальнейшего использования

###

 Команда             | Описание
:--------------------|:----------
`tmux new-session -t [existing session] -s [new session]` | Создает подключение к групповому сеансу.
`tmux show-messages` | Отображает журнал сообщений в текущем окне, полезно для отладка.
`tmux -S [socket]` | Создает новый сеанс, используя сокет вместо имени.
`tmux -S [socket] attach` | Присоединяется к существующему сеансу, используя сокет вместо имени.

#### Ссылки

12. https://www.virtualbox.org/

13. https://www.vagrantup.com/docs/getting-started/

14. https://tmate.io/

15. https://tmate.io/